## Collapse Board Data to School Level, Merge with IPEDS and NACUBO Data

### For Tables 3 and 4 of Elite Embeddedness

By Charlie Eaton and Albina Gibadullina

In [ ]:
use unitid firstname middlename lastname suffix undergrad mba jd md phd otherprofdegree ///
alumni schoolname schooltype gender industry_code financetype boardofficer boardchair unitid ///
firmid firmname_new organization forbes forbesyear eigenvector betweenness strength using board_members_all_long20220501, clear

In [62]:
use "/Users/Charlie/Dropbox/Bankers in the Ivory Tower Book/2 Our New Financial Oligarchy/Data/Boards 2000 to 2017/board_members_all_long20220501.dta", clear

. use "/Users/Charlie/Dropbox/Bankers in the Ivory Tower Book/2 Our New Financi
> al Oligarchy/Data/Boards 2000 to 2017/board_members_all_long20220501.dta", cl
> ear


In [63]:
drop if schooltype=="liberal arts"
drop if unitid==232186 | unitid==126818 | unitid==209542 | unitid==221759 | unitid==102614 | unitid==129020 | unitid==181464
drop if year<2003


(1,886 observations deleted)

(1,689 observations deleted)

(4,765 observations deleted)


In [64]:
collapse (sum) pe hedge otherfinance nonfinance (first) institutionname schoolname schooltype , by(unitid year)
gen schooltypenum =2 if schooltype=="public"
replace schooltypenum=3 if schooltype=="research"
label define schooltype 2 "public" 3 "private"
label values schooltypenum schooltype



(440 missing values generated)

(440 real changes made)




In [65]:
merge 1:1 unitid year using d_nacubo_funds_2003_2017, gen(b_merge) keep(1 3)
**correct university of delaware unitid to flagship
replace unitid=130943 if unitid==130343



    Result                      Number of obs
    -----------------------------------------
    Not matched                            22
        from master                        22  (b_merge==1)
        from using                          0  (b_merge==2)

    Matched                               865  (b_merge==3)
    -----------------------------------------

(15 real changes made)


In [66]:
merge 1:1 unitid year using d_f1a_endow_ipeds_2003_2018, nogen keep(1 3)


    Result                      Number of obs
    -----------------------------------------
    Not matched                           471
        from master                       471  
        from using                          0  

    Matched                               416  
    -----------------------------------------


In [68]:
foreach file in board1_forbes_400_count board2_forbes_400_count ///
mba_forbes_400_count ba_forbes_400_count {
 qui   merge 1:1 unitid year using d_`file', gen(m`file') keep(1 3)
}

foreach var in pehedge otherfin technology forbes {
qui    egen `var'board=rowtotal(`var'board1 `var'board2)
}

foreach var in pehedge otherfin technology forbes {
  qui  egen alumni`var'=rowtotal(`var'_ba `var'_mba)
}

In [69]:
cd "/Users/Charlie/Dropbox/Bankers in the Ivory Tower Book/2 Our New Financial Oligarchy/Data/Boards 2000 to 2017"
merge 1:1 unitid year using nonmissing_sat_act_admissions_all_universities_20181223, nogen keep(1 3) keepusing(acceptrate)


/Users/Charlie/Dropbox/Bankers in the Ivory Tower Book/2 Our New Financial Oligarchy/Data/Boards 2000 to 2017


    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                               887  
    -----------------------------------------


In [70]:
cd "/Users/Charlie/Google Drive/Lincoln Project/IPEDS Disaggregated/Data"
*merge 1:1 unitid year using hd_flags_1981_2018, keepusing(instnm control fips) keep(1 3)
merge 1:1 unitid year using finaid_2000_2017, keepusing(fgrnt_p) nogen keep(1 3)

merge 1:1 unitid year using fteenrollment1981_to_2018, nogen ///
keepusing(fte_count ftfirsteftotlt ptfirsteftotlt ftfirstefasiat ptfirstefasiat ftfirstefwhitt ptfirstefwhitt) ///
keep(1 3)

**change rutgers unitid for years that it reported revenue at different level
replace unitid=186362 if unitid==186380 & year<2005
merge 1:1 unitid year using financial_all_1981_2017, ///
keepusing(total03_revenue) keep(1 3)

replace unitid=186380 if unitid==186362

merge m:1 year using cpi_scalar2017, nogen keep(1 3)


/Users/Charlie/Google Drive/Lincoln Project/IPEDS Disaggregated/Data


    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                               887  
    -----------------------------------------


    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                               887  
    -----------------------------------------

(2 real changes made)


    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                               887  (_merge==3)
    -----------------------------------------

(2 real changes made)


    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                    

In [71]:
rename EndowmentFunds endowment_assets
replace endowment_assets=endowment_public_ipeds if endowment_assets==.



(17 real changes made)


In [72]:
cd "/Users/Charlie/Dropbox/Stata_Notebooks/highfinancesocialcircuitry/data"

/Users/Charlie/Dropbox/Stata_Notebooks/highfinancesocialcircuitry/data


In [73]:
ds

unitid        otherfinance  schooltype    Institution   pehedgeboa~1  forbesboard2  mboard2_fo~t  technolo~mba  otherfin_ba   otherfinbo~d  alumniothe~n  fgrnt_p       ptfirstefa~t  total03_re~e
year          nonfinance    schooltype~m  b_merge       otherfinbo~1  pehedgeboa~2  forbes_mba    mmba_forbe~t  technolo~_ba  technology~d  alumnitech~y  ftfirsteft~t  ftfirstefw~t  _merge
pe            institutio~e  endowment~ts  endowment~ds  technology~1  otherfinbo~2  pehedge_mba   forbes_ba     mba_forbes~t  forbesboard   alumniforbes  ptfirsteft~t  ptfirstefw~t  cpi_sca~2017
hedge         schoolname    State         forbesboard1  mboard1_fo~t  technology~2  otherfin_mba  pehedge_ba    pehedgeboard  alumnipehe~e  acceptrate    ftfirstefa~t  fte_count


In [74]:
save d_school_level_elite_embed, replace

file d_school_level_elite_embed.dta saved


In [72]:
save d_school_level_elite_embed, replace

(file d_school_level_elite_embed.dta not found)
file d_school_level_elite_embed.dta saved


In [73]:
tab year


       year |      Freq.     Percent        Cum.
------------+-----------------------------------
       2003 |         55        6.20        6.20
       2004 |         57        6.43       12.63
       2005 |         57        6.43       19.05
       2006 |         59        6.65       25.70
       2007 |         59        6.65       32.36
       2008 |         60        6.76       39.12
       2009 |         60        6.76       45.89
       2010 |         60        6.76       52.65
       2011 |         60        6.76       59.41
       2012 |         60        6.76       66.18
       2013 |         60        6.76       72.94
       2014 |         60        6.76       79.71
       2015 |         60        6.76       86.47
       2016 |         60        6.76       93.24
       2017 |         60        6.76      100.00
------------+-----------------------------------
      Total |        887      100.00


In [74]:
ds

unitid        pe            otherfinance  institutio~e  schooltype    State         b_merge       instnm        control       fgrnt_p       ptfirsteft~t  ptfirstefa~t  ptfirstefw~t  endowment02   finmerge
year          hedge         nonfinance    schoolname    EndowmentF~s  Institution   acceptrate    fips          _merge        ftfirsteft~t  ftfirstefa~t  ftfirstefw~t  fte_count     total03_re~e


In [78]:
sum EndowmentFunds if year==2003
list unitid schoolname instnm year if EndowmentFunds<50000



    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
EndowmentF~s |         54     2437307     3312812      43699   1.88e+07


     +----------------------------------------------------------------------------+
     | unitid                             schoolname                instnm   year |
     |----------------------------------------------------------------------------|
593. | 196097   state university of new york buffalo   suny at stony brook   2003 |
     +----------------------------------------------------------------------------+


In [147]:
set more off
sort unitid year
keep if b_merge==1
outsheet unitid schoolname year using schools_without_match.csv, replace comma




(0 observations deleted)



In [6]:
merge 1:1 unitid year using ///
"/Users/Charlie/Dropbox/Bankers in the Ivory Tower Book/2 Our New Financial Oligarchy/Data/Boards 2000 to 2017/board_endow_donation_directory_20181221.dta"

variable unitid not found


r(111);


In [82]:
use "/Users/Charlie/Dropbox/Bankers in the Ivory Tower Book/2 Our New Financial Oligarchy/Data/Boards 2000 to 2017/board_endow_donation_directory_20181221.dta", clear

In [85]:
tab schooltypenum


     (first) |
  schooltype |      Freq.     Percent        Cum.
-------------+-----------------------------------
liberal arts |         58        4.70        4.70
      public |        646       52.39       57.10
    research |        529       42.90      100.00
-------------+-----------------------------------
       Total |      1,233      100.00


In [39]:
tabstat EndowmentFunds, s(mean) by(year)


Summary for variables: EndowmentFunds
Group variable: year 

    year |      Mean
---------+----------
    2003 |  322489.5
    2004 |  361304.1
    2005 |  400637.7
    2006 |  444551.3
    2007 |  523789.1
    2008 |  521912.9
    2009 |  3.64e+08
    2010 |  3.97e+08
    2011 |  9.86e+08
    2012 |  9.69e+08
    2013 |  3.44e+07
    2014 |  614731.4
    2015 |  660258.8
    2016 |  641273.9
    2017 |  704576.6
---------+----------
   Total |  1.85e+08
--------------------


In [11]:
replace EndowmentFunds=EndowmentFunds000 if EndowmentFunds==.
replace EndowmentFunds=EndowmentFunds2009 if EndowmentFunds==.
replace EndowmentFunds=Funds000 if EndowmentFunds==.
*replace EndowmentFunds=(000) if EndowmentFunds==.
replace EndowmentFunds=Funds if EndowmentFunds==.
tab year if EndowmentFunds!=.


(0 real changes made)

(0 real changes made)

(0 real changes made)

(0 real changes made)


       year |      Freq.     Percent        Cum.
------------+-----------------------------------
       2003 |        714        5.89        5.89
       2004 |        739        6.09       11.98
       2005 |        746        6.15       18.13
       2006 |        765        6.31       24.44
       2007 |        785        6.47       30.91
       2008 |        791        6.52       37.43
       2009 |        842        6.94       44.37
       2010 |        827        6.82       51.19
       2011 |        816        6.73       57.91
       2012 |        813        6.70       64.62
       2013 |        946        7.80       72.42
       2014 |        896        7.39       79.80
       2015 |        816        6.73       86.53
       2016 |        816        6.73       93.26
       2017 |        818        6.74      100.00
------------+-----------------------------------
      Total |     12,130

In [199]:
replace Institution=InstitutionName if Institution==""
replace Institution=INSTITUTION if Institution==""
drop if Institution==""


(0 observations deleted)

(795 real changes made)

(842 real changes made)

(29 observations deleted)


In [96]:
codebook Institution if year==2012


--------------------------------------------------------------------------------
Institution                                                          Institution
--------------------------------------------------------------------------------

                  Type: String (str92), but longest is str84

         Unique values: 806                       Missing "": 0/806

              Examples: "Denison University "
                        "Linfield College "
                        "Rensselaer Polytechnic Institute "
                        "UC Davis Foundation "



(8 vars, 896 obs)


In [ ]:
replace Institution=InstitutionName if Institution==""
replace Institution=INSTITUTION if Institution==""
replace Institution=A if if year==2008 | year==2007 | year==2010
drop if Institution==""
tab year

In [40]:
qui {
    use "IPEDS and Classification IDFILE.dta", clear

destring ipeds, replace

replace ipeds=124557 if unitid==18088
replace ipeds=128300 if unitid==20246
replace ipeds=149587 if unitid==20786
replace ipeds=170976 if unitid==20347
replace ipeds=214661 if unitid==20146
replace ipeds=214661 if unitid==22369
replace ipeds=221470 if unitid==20257
replace ipeds=166665 if unitid==20252
replace ipeds=103529 if unitid==20250
replace ipeds=110501 if unitid==17945
replace ipeds=188030 if unitid==19023
replace ipeds=174066 if unitid==19948
replace ipeds=195827 if unitid==31586
replace ipeds=110501 if unitid==20777
replace ipeds=228732 if unitid==19729
replace ipeds=229090 if unitid==19748
replace ipeds=229090 if unitid==20784
replace ipeds=149587 if unitid==20248
replace ipeds=190035 if unitid==22367
replace ipeds=439154 if unitid==19741
replace ipeds=178439 if unitid==18927
replace ipeds=229407 if unitid==20543
replace ipeds=124557 if unitid==20778

sort unitid
}

merge 1:m unitid using "Merced Enrollment, Endowment, Expenditures.dta"", nogen

qui {
    
drop if Year==.
sort unitid Year
rename Year year
drop if year<2002

encode Control, gen(control)

drop if control>2
rename control cae_control
gen control=1 if cae_control==2
replace control=2 if cae_control==1

rename unitid caeid
rename ipeds unitid

drop if unitid==.
**Drop Penn State Duplicates**
drop if caeid==22369
**Drop New Mexico State Duplicates**
drop if caeid==31571

gen order=1

sort unitid order year
}
ds



. merge 1:m unitid using "Merced Enrollment, Endowment, Expenditures.dta", nogen

    Result                      Number of obs
    -----------------------------------------
    Not matched                         1,231
        from master                     1,228  
        from using                          3  

    Matched                            48,598  
    -----------------------------------------

. qui {

. ds
caeid         unitid        unitName      city          state         zip           Control       className     year          Headcount     Endowment     Expenditures  FTE_Enroll~t  cae_control   control       order


In [45]:
use "/Users/Charlie/Google Drive/Lincoln Project/IPEDS Disaggregated/Data/nacubo_all_2000_2013", clear

. use "/Users/Charlie/Google Drive/Lincoln Project/IPEDS Disaggregated/Data/nac
> ubo_all_2000_2013", clear


In [47]:
tab year if Endowment_Funds_no_annuity!=.


       year |      Freq.     Percent        Cum.
------------+-----------------------------------
       2000 |        437        4.37        4.37
       2001 |        452        4.52        8.89
       2002 |        651        6.51       15.41
       2003 |        760        7.60       23.01
       2004 |        779        7.79       30.80
       2005 |        790        7.90       38.71
       2006 |        793        7.93       46.64
       2007 |        781        7.81       54.45
       2008 |        741        7.41       61.86
       2009 |        785        7.85       69.72
       2010 |        761        7.61       77.33
       2011 |        748        7.48       84.81
       2012 |        755        7.55       92.37
       2013 |        763        7.63      100.00
------------+-----------------------------------
      Total |      9,996      100.00


In [41]:
merge 1:1 unitid order year ///
using "/Users/Charlie/Google Drive/Lincoln Project/IPEDS Disaggregated/Data/nacubo_all_2000_2013", ///
gen(nacubo_merge) update

. merge 1:1 unitid order year using "/Users/Charlie/Google Drive/Lincoln Projec
> t/IPEDS Disaggregated/Data/nacubo_all_2000_2013", gen(nacubo_merge) update
(variable year was int, now float to accommodate using data's values)

    Result                      Number of obs
    -----------------------------------------
    Not matched                        11,501
        from master                     8,172  (nacubo_merge==1)
        from using                      3,329  (nacubo_merge==2)

    Matched                             6,667
        not updated                     6,667  (nacubo_merge==3)
        missing updated                     0  (nacubo_merge==4)
        nonmissing conflict                 0  (nacubo_merge==5)
    -----------------------------------------


In [44]:
set line 255
sort year
list year unitid upper_inst unitName Endowment Endowment_Funds_no_annuity if regexm(upper_inst, "HAWA")==1





       +-----------------------------------------------------------------------------------+
       | year   unitid                        upper_inst   unitName   Endowm~t   Endowme~y |
       |-----------------------------------------------------------------------------------|
  306. | 2000   141839              UNIVERSITY OF HAWAII                     .           0 |
  501. | 2001   141839              UNIVERSITY OF HAWAII                     .           0 |
 1991. | 2003   141839              UNIVERSITY OF HAWAII                     .   152111000 |
 3632. | 2004   141839              UNIVERSITY OF HAWAII                     .   176806000 |
 5459. | 2005   141839              UNIVERSITY OF HAWAII                     .   186982000 |
       |-----------------------------------------------------------------------------------|
 9896. | 2009   141963   UNIVERSITY OF HAWAII FOUNDATION                     .   158673840 |
10455. | 2010   141963   UNIVERSITY OF HAWAII FOUNDATION          

In [35]:
codebook unitName


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
unitName                                                                                                                                                                                                                                               unitName
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

                  Type: String (str77), but longest is str64

         Unique values: 1,423                     Missing "": 7,954/22,793

              Examples: ""
                        "Brewton-Parker College"
                

In [14]:
use "/Users/Charlie/Google Drive/Lincoln Project/IPEDS Disaggregated/Data/nacubo_all_1990_2013", clear
tab year if Endowment_Funds!=.




       year |      Freq.     Percent        Cum.
------------+-----------------------------------
       2003 |        650        8.27        8.27
       2004 |        672        8.55       16.83
       2005 |        686        8.73       25.56
       2006 |        705        8.97       34.53
       2007 |        716        9.11       43.65
       2008 |        728        9.27       52.91
       2009 |        751        9.56       62.47
       2010 |        733        9.33       71.80
       2011 |        732        9.32       81.12
       2012 |        737        9.38       90.50
       2013 |        746        9.50      100.00
------------+-----------------------------------
      Total |      7,856      100.00


In [8]:
list Year unitName unitid Endowment if state=="HI"


       +--------------------------------------------------------------+
       | Year                           unitName   unitid   Endowment |
       |--------------------------------------------------------------|
  815. | 1969   Chaminade University of Honolulu    18286           . |
 1711. | 1970   Chaminade University of Honolulu    18286           . |
 2584. | 1971   Chaminade University of Honolulu    18286           . |
 3630. | 1972   Chaminade University of Honolulu    18286           . |
 5580. | 1974   Chaminade University of Honolulu    18286       83667 |
       |--------------------------------------------------------------|
 6416. | 1975   Chaminade University of Honolulu    18286       89712 |
 7347. | 1976   Chaminade University of Honolulu    18286       95690 |
 8414. | 1977   Chaminade University of Honolulu    18286       39187 |
 9465. | 1978                 Hawaii Loa College    19995   127172172 |
10353. | 1979                 Hawaii Loa College    19995      

In [17]:
use instnm unitid year using hd_flags_1981_2018, clear

merge 1:1 unitid year using financial_all_1981_2017, gen(finmerge) ///
keepusing(total03_revenue) keep(1 3)



(variable unitid was long, now double to accommodate using data's values)

    Result                      Number of obs
    -----------------------------------------
    Not matched                       135,569
        from master                   135,569  (finmerge==1)
        from using                          0  (finmerge==2)

    Matched                           169,543  (finmerge==3)
    -----------------------------------------


In [20]:
set line 255
list instnm unitid year total03_revenue if regexm(instnm, "RUTGERS")==1 & year>2002 & year<2005




        +------------------------------------------------------------------------+
        |                                      instnm   unitid   year   total0~e |
        |------------------------------------------------------------------------|
128018. | RUTGERS THE STATE UNIVERSITY CENTRAL OFFICE   186362   2003   1.31e+09 |
128019. | RUTGERS THE STATE UNIVERSITY CENTRAL OFFICE   186362   2004   1.41e+09 |
128042. |                   RUTGERS UNIVERSITY-CAMDEN   186371   2003          . |
128043. |                   RUTGERS UNIVERSITY-CAMDEN   186371   2004          . |
128078. |            RUTGERS UNIVERSITY-NEW BRUNSWICK   186380   2003          . |
        |------------------------------------------------------------------------|
128079. |            RUTGERS UNIVERSITY-NEW BRUNSWICK   186380   2004          . |
128114. |                   RUTGERS UNIVERSITY-NEWARK   186399   2003          . |
128115. |                   RUTGERS UNIVERSITY-NEWARK   186399   2004          . |
 